In [6]:
import os
os.chdir("/home/patrick/Desktop/VeriNet")

import torch
import numpy as np
import torch.nn as nn

from src.neural_networks.verinet_nn import VeriNetNN
from src.neural_networks.cifar10 import Cifar10
from src.data_loader.nnet import NNET

%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


### Convert Cifar10 to nnet

In [2]:
model = Cifar10()
model.load("./data/models_torch/cifar10_state_dict.pth")
nnet = NNET()

img_size = 32*32

input_mean = np.zeros(3*32*32)
input_mean[:32*32] = 0.4914
input_mean[32*32:2*32*32] = 0.4822
input_mean[2*32*32:] = 0.4465

input_range = np.zeros(3*32*32)
input_range[:32*32] = 0.2023
input_range[32*32:2*32*32] = 0.1994
input_range[2*32*32:] = 0.2010

nnet.init_nnet_from_verinet_nn(model=model, input_shape=np.array((3, 32, 32)), min_values=0.0, 
                               max_values=1.0, input_mean=input_mean, input_range=input_range)
nnet.write_nnet_to_file("./data/models_nnet/cifar10_conv.nnet")

### Convert eran to nnet

Converts the mnist Sigmoid/Tanh networks from eran to nnet format

In [17]:
model_name = "ffnnTANH__PGDK_w_0.1_6_500"
model_path = f"/home/patrick/Desktop/VeriNet/eran-benchmark/eran/data/{model_name}.pyt"
act_func = nn.Tanh

layers = [
    nn.Sequential(nn.Linear(784, 500), act_func()),
    nn.Sequential(nn.Linear(500, 500), act_func()),
    nn.Sequential(nn.Linear(500, 500), act_func()),
    nn.Sequential(nn.Linear(500, 500), act_func()),
    nn.Sequential(nn.Linear(500, 500), act_func()),
    nn.Sequential(nn.Linear(500, 500), act_func()),
    nn.Sequential(nn.Linear(500, 10)),
    ]

with open(model_path, "r") as file:
    # skip header
    file.readline()
    file.readline()

    for j in range(len(layers)):
        layer = list(layers[j].children())[0]
        in_size = layer.in_features
        out_size = layer.out_features
        weights = torch.Tensor([float(w) for w in file.readline().replace('[', '').replace(']', '').split(", ")])
        layer.weight.data = weights.reshape(out_size, in_size)
        bias = torch.Tensor([float(w) for w in file.readline().replace('[', '').replace(']', '').split(", ")])
        layer.bias.data = bias

        file.readline()

model = VeriNetNN(layers)
nnet = NNET()
nnet.init_nnet_from_verinet_nn(model=model, input_shape=np.array((784)), min_values=0.0, 
                               max_values=1.0, input_mean=0.1307, input_range=0.3081)
nnet.write_nnet_to_file(f"./data/models_nnet/{model_name}.nnet")